# Riccardo Romanello

Please, do not be too harsh with me for this notebook.
Have been a couple of hard weeks. 

However, I prepared this simple test. (also provided requirements.txt file) 

First, the import.

In [17]:
import pennylane as qml
from transform.routing_transform import apply_routing
import networkx as nx

Now that the import have been resolved, we move to the actual code. 
First, we declare the topology. 

As written in the readme, I assume it to be with $n$ nodes, with names from $0$ to $n-1$. Moreover, the graph must be connected. It is interpreted as non-oriented (undirected).

In [18]:
connectivity = {
    0 : {1},
    1 : {0, 2},
    2 : {1, 3},
    3 : {2}
}

# a simple line of 4 nodes

Now, a simple circuit to test the efficiency and correctness of my method. 

In [19]:
dev = qml.device("default.qubit", wires=6)

@qml.qnode(dev)
def circuit():
    qml.PauliX(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.PauliX(wires=0)
    return qml.state()


We print the circuit before compiling it. 

In [20]:
print("Circuit before compilation")
print(qml.draw(circuit)())

Circuit before compilation
0: ──X─╭●──X─┤  State
1: ────╰X────┤  State


Now, it's routing time. 
First, we execute the circuit so that we can retrieve the _tape property. 
We provide to the method apply_routing the connectivity and the mapping strategy as well. 
If you want to change strategy, use "majority" for the majority mapping or "max_pairs" for the maximum interacting pairs.

With the random strategy, you can execute the following box and keep finding different results. 
Usually I get only a single swap. It suffices to create a bigger topology graph to create more swaps. 

Subsequently, we create a new circuit and print it.

In [30]:
circuit()
compiled, measurements, qubits = apply_routing(circuit._tape, nx.Graph(connectivity), "random")
# compiled, measurements, qubits = apply_routing(circuit._tape, nx.Graph(connectivity), "majority")
# compiled, measurements, qubits = apply_routing(circuit._tape, nx.Graph(connectivity), "max_pairs")

second_device = qml.device("default.qubit", wires=qubits)
@qml.qnode(second_device)
def new_circuit():
    for op in compiled:
        qml.apply(op)
    return qml.state()

print("Circuit after compilation")
print(qml.draw(new_circuit)())

Circuit after compilation
0: ──X─╭SWAP─────────────┤  State
1: ────│─────╭SWAP───────┤  State
2: ────╰SWAP─│─────╭●──X─┤  State
3: ──────────╰SWAP─╰X────┤  State
